In [3]:
import open3d as o3d
import numpy as np
import copy
import time
import tempfile

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [9]:



def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def save_registration_result(source, transformation):
    source_temp = copy.deepcopy(source)
    temp_file = tempfile.NamedTemporaryFile(suffix=".ply", delete=False)
    registered = source_temp.transform(transformation)
    o3d.io.write_point_cloud(temp_file.name , registered, write_ascii=True)
    return temp_file.name
    


def prepare_data(source_path, target_path):
    pcd_data = o3d.data.DemoICPPointClouds()
    source = o3d.io.read_point_cloud(source_path)
    target = o3d.io.read_point_cloud(target_path)
    print("Visualization of two point clouds before manual alignment")
    draw_registration_result(source, target, np.identity(4))
    return source, target


def pick_points(pcd):
    print("")
    print(
        "1) Please pick at least three correspondences using [shift + left click]"
    )
    print("   Press [shift + right click] to undo point picking")
    print("2) After picking points, press 'Q' to close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()

def scalefactor_from_correspondencies(source_point_coordinates, target_point_coordinates):

    def dist(p1,p2):
        
        # Compute Euclidean distances between corresponding points
        distances = np.linalg.norm(p1 - p2, axis=1)
        
        # Return the average distance
        return np.mean(distances)
    
   
    source_points = np.asarray(source_point_coordinates)
    target_points = np.asarray(target_point_coordinates)


    scale_factor = dist(source_points,target_points)
    center = np.mean(source_point_coordinates, axis=0)


    print("Type of input: ", type(source_point_coordinates))
    print("Source Coordinates Listed")
    for corr in source_point_coordinates: print(corr)
    
    print("Target Coordinates Listed")
    for corr in target_point_coordinates: print(corr)

    return scale_factor, center





def register_via_correspondences(source, target, source_points, target_points):
    corr = np.zeros((len(source_points), 2))
    corr[:, 0] = source_points
    corr[:, 1] = target_points

    ## Verrechung skalierung

    # die transformationsmatrix, die Cloudcompare errechnet lautet:

    # [ -4.753,  1.162, -9.471, 3.797],
    # [ 3.121,  10.188, -0.316, -12.449],
    # [ 9.017, -2.914, -4,882, -12.342], 
    # [ 0.000, 0.000, 0.000, 1.000] 

    # darin ist die skalierung vom factor 10.6597 integriert
   
    #Rescaling the lidar to the size of the Gaussian
    # source_point_coordinates = [np.asarray(source.points)[i] for i in source_points]
    # target_point_coordinates = [np.asarray(target.points)[i] for i in target_points]



    #scale_factor, center = scalefactor_from_correspondencies(source_point_coordinates, target_point_coordinates)

    #scaled_source = scale_factor * (source - center) + center


    # estimate rough transformation using correspondences
    print("Compute a rough transform using the correspondences given by user")
    p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    trans_init = p2p.compute_transformation(source, target,
                                            o3d.utility.Vector2iVector(corr))
    
    #override trans init with transform from cloud compare to show if it is working
    trans_init = np.array([
        [-4.753,  1.162, -9.471,  3.797],
        [ 3.121, 10.188, -0.316, -12.449],
        [ 9.017, -2.914, -4.882, -12.342],
        [ 0.000,  0.000,  0.000,   1.000]
    ])

    
    # point-to-point ICP for refinement
    print("Perform point-to-point ICP refinement")
    threshold = 0.3  # 3cm distance threshold
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    temp_registration = save_registration_result(source, reg_p2p.transformation)
    draw_registration_result(source, target, reg_p2p.transformation)
    return temp_registration

def downsample_point_cloud(pcd, voxel_size):
    
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)
   
    return pcd_down

def demo_manual_registration():

    print("Demo for manual ICP")
    pcd_data = o3d.data.DemoICPPointClouds()
    source, target = prepare_data(pcd_data.paths[0], pcd_data.paths[2])

    # pick points from two point clouds and builds correspondences
    source_points = pick_points(source)
    target_points = pick_points(target)
    assert (len(source_points) >= 3 and len(target_points) >= 3)
    assert (len(source_points) == len(target_points))
    register_via_correspondences(source, target, source_points, target_points)
    print("")

def manual_registration(lidar_path, gaussiansplat_path):

    print("manual ICP")
    
    source, target = prepare_data(lidar_path, gaussiansplat_path)

    lidar_points = len(source.points)
    gaussian_points = len(target.points)

    voxel_size = (lidar_points//gaussian_points)*0.001
    print(voxel_size)
    source_down = downsample_point_cloud(source, voxel_size)



    # pick points from two point clouds and builds correspondences
    source_points = pick_points(source_down)
    print("Type of Points: ",type(source_points))
    picked_coordinates = [source.points[i] for i in source_points]
    print("Coordinates of picked points:")
    for coord in picked_coordinates:
        print(coord)
    
    target_points = pick_points(target)
    assert (len(source_points) >= 3 and len(target_points) >= 3)
    assert (len(source_points) == len(target_points))
    registered = register_via_correspondences(source_down, target, source_points, target_points)
    print("")
    return registered




In [10]:
cc_pc = "D:\MASTERPROJEKT\Daten\musikzimmer\gaussiansplat\iteration_30000\musikzimmer_gaussiansplat_cc.ply"
lidar_pc = "D:\MASTERPROJEKT\Daten\musikzimmer\lidar\Musikzimmer_iphone_LIDAR_v02_cc.ply"
 

cc_pc_alligned = manual_registration(lidar_pc, cc_pc)

manual ICP
Visualization of two point clouds before manual alignment
0.001
:: Downsample with a voxel size 0.001.

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #1088858 (0.87, 0.74, 1.2) to add in queue.
[Open3D INFO] Picked point #1020462 (2.4, 0.39, 1.1) to add in queue.
[Open3D INFO] Picked point #1106481 (0.75, -0.23, 1.2) to add in queue.
[Open3D INFO] Picked point #721483 (2.3, -0.75, 1.1) to add in queue.

Type of Points:  <class 'list'>
Coordinates of picked points:
[-0.99639  -0.60068   1.102815]
[ 0.850536 -1.161605 -0.612872]
[-3.297311  0.318845  1.108831]
[ 3.571142 -0.562967  1.145041]

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #4237 (-6.5, -1.1, 31